## Missing Value Imputation - FancyImputer -KNN

refer : https://pypi.org/project/fancyimpute/

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [8]:
df1=pd.read_csv("Without_encoded.csv")
df1.drop('Unnamed: 0',axis=1,inplace=True)
df1.head()

In [99]:
data=df1[['USERYEARSOFEXPERIENCE','AGEGROUP','COMPANYINDUSTRY','OCCUPATION','MEDIANINCOME','HIGHESTEDUCATIONLEVEL']].head(100)

In [103]:
data.rename(columns={'USERYEARSOFEXPERIENCE':'EXPERIENCE','MEDIANINCOME':'SALARY'},inplace=True)

In [104]:
data.head(10)

,EXPERIENCE,AGEGROUP,COMPANYINDUSTRY,OCCUPATION,SALARY,HIGHESTEDUCATIONLEVEL
0,0.0,18-24,NaN,NaN,11523.0,GED
1,1.0,18-24,RETAIL,STOCKERS AND ORDER FILLERS,NaN,NaN
2,0.0,< 18,NaN,NaN,NaN,NaN
3,1.0,18-24,CONSTRUCTION,Others,51885.0,BACHELOR OF ARTS
4,6.0,25-34,NaN,NaN,62400.0,SOME COLLEGE (NO DEGREE)
5,0.0,< 18,FOOD AND BEVERAGE,WAITERS AND WAITRESSES,15169.0,ASSOCIATE OF ARTS
6,4.0,18-24,NaN,RETAIL SALESPERSONS,26325.0,Others
7,1.0,18-24,NaN,NaN,41447.0,NaN
8,0.0,18-24,CHILDCARE,NANNIES,28452.0,NaN
9,0.0,18-24,CUSTOMER SERVICE,"COMMUNITY AND SOCIAL SERVICE SPECIALISTS, ALL ...",16331.0,NaN


In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   EXPERIENCE             100 non-null    float64
 1   AGEGROUP               100 non-null    object 
 2   COMPANYINDUSTRY        51 non-null     object 
 3   OCCUPATION             73 non-null     object 
 4   SALARY                 84 non-null     float64
 5   HIGHESTEDUCATIONLEVEL  81 non-null     object 
dtypes: float64(2), object(4)
memory usage: 4.8+ KB


Based on the information we have, here is our situation:
* Categorical data with text that needs encoded: AGEGROUP,COMPANYINDUSTRY,OCCUPATION,HIGHESTEDUCATIONLEVEL
* Categorical data that has null values: COMPANYINDUSTRY,OCCUPATION,HIGHESTEDUCATIONLEVEL

In [114]:
from sklearn.preprocessing import OrdinalEncoder
from fancyimpute import KNN

encoder = OrdinalEncoder(dtype=object)
imputer = KNN()

### 1. We encode the categorical data 

In [125]:
# create a list of categorical columns to iterate over
cat_cols = ['AGEGROUP','COMPANYINDUSTRY','OCCUPATION','HIGHESTEDUCATIONLEVEL']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(data[columns])

/home/azure/.pyenv/versions/mlflow_env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/azure/.pyenv/versions/mlflow_env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/azure/.pyenv/versions/mlflow_env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

### 2. We can impute the data, convert the data back to a DataFrame and add back in the column names 

In [126]:
data

,EXPERIENCE,AGEGROUP,COMPANYINDUSTRY,OCCUPATION,SALARY,HIGHESTEDUCATIONLEVEL
0,0.0,0,NaN,NaN,11523.0,5
1,1.0,0,13,12,NaN,NaN
2,0.0,6,NaN,NaN,NaN,NaN
3,1.0,0,5,8,51885.0,2
4,6.0,1,NaN,NaN,62400.0,10
...,...,...,...,...,...,...
95,4.0,0,13,11,14207.0,10
96,3.0,0,NaN,8,62657.0,3
97,22.0,3,11,0,156424.0,7
98,3.0,0,NaN,NaN,17564.0,0


In [127]:
# impute data and convert 
encode_data = pd.DataFrame(np.round(imputer.fit_transform(data)),columns = data.columns)

Imputing row 1/100 with 2 missing, elapsed time: 0.003


In [128]:
encode_data.head(10)

,EXPERIENCE,AGEGROUP,COMPANYINDUSTRY,OCCUPATION,SALARY,HIGHESTEDUCATIONLEVEL
0,0.0,0.0,10.0,9.0,11523.0,5.0
1,1.0,0.0,13.0,12.0,36709.0,0.0
2,0.0,6.0,8.0,5.0,26904.0,2.0
3,1.0,0.0,5.0,8.0,51885.0,2.0
4,6.0,1.0,8.0,8.0,62400.0,10.0
5,0.0,6.0,8.0,13.0,15169.0,0.0
6,4.0,0.0,9.0,10.0,26325.0,9.0
7,1.0,0.0,9.0,10.0,41447.0,6.0
8,0.0,0.0,4.0,7.0,28452.0,6.0
9,0.0,0.0,6.0,2.0,16331.0,8.0


In [137]:
def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].dropna().unique(), 0)}

def integer_encode(df , variable, ordinal_mapping):
    df[variable] = df[variable].map(ordinal_mapping)

mappin = dict()
def imputation(df1 , cols):
    df = df1.copy()
    #Encoding dict &amp; Removing nan    
    #mappin = dict()
    for variable in cols:
        mappings = find_category_mappings(df, variable)
        mappin[variable] = mappings

    #Apply mapping
    for variable in cols:
        integer_encode(df, variable, mappin[variable])  

    #Minmaxscaler and KNN imputation 
    sca = mm.fit_transform(df)
    knn_imputer = KNNImputer(n_neighbors=7)
    knn = knn_imputer.fit_transform(sca)
    df.iloc[:,:] = mm.inverse_transform(knn)
    for i in df.columns : 
        df[i] = round(df[i]).astype('int')

    #Inverse transform
    for i in cols:
        inv_map = {v: k for k, v in mappin[i].items()}
        df[i] = df[i].map(inv_map)
    return df




In [138]:
#Imputation
g=imputation(data,['COMPANYINDUSTRY','OCCUPATION','HIGHESTEDUCATIONLEVEL'])
g.head(10)

,EXPERIENCE,AGEGROUP,COMPANYINDUSTRY,OCCUPATION,SALARY,HIGHESTEDUCATIONLEVEL
0,0,0,5,10,11523,5
1,1,0,13,12,33703,9
2,0,6,1,7,36723,6
3,1,0,5,8,51885,2
4,6,1,6,8,62400,10
5,0,6,8,13,15169,0
6,4,0,4,10,26325,9
7,1,0,5,7,41447,9
8,0,0,4,7,28452,9
9,0,0,6,2,16331,9


In [119]:
encode_data.head(10)

,EXPERIENCE,AGEGROUP,COMPANYINDUSTRY,OCCUPATION,SALARY,HIGHESTEDUCATIONLEVEL
0,0.0,0.0,10.0,9.0,11523.0,5.0
1,1.0,0.0,13.0,12.0,36709.0,0.0
2,0.0,6.0,8.0,5.0,26904.0,2.0
3,1.0,0.0,5.0,8.0,51885.0,2.0
4,6.0,1.0,8.0,8.0,62400.0,10.0
5,0.0,6.0,8.0,13.0,15169.0,0.0
6,4.0,0.0,9.0,10.0,26325.0,9.0
7,1.0,0.0,9.0,10.0,41447.0,6.0
8,0.0,0.0,4.0,7.0,28452.0,6.0
9,0.0,0.0,6.0,2.0,16331.0,8.0


In [131]:
df2=df1[['USERYEARSOFEXPERIENCE','AGEGROUP','COMPANYINDUSTRY','OCCUPATION','MEDIANINCOME','HIGHESTEDUCATIONLEVEL']].head(100)

In [132]:
df2.head(10)

,USERYEARSOFEXPERIENCE,AGEGROUP,COMPANYINDUSTRY,OCCUPATION,MEDIANINCOME,HIGHESTEDUCATIONLEVEL
0,0.0,18-24,NaN,NaN,11523.0,GED
1,1.0,18-24,RETAIL,STOCKERS AND ORDER FILLERS,NaN,NaN
2,0.0,< 18,NaN,NaN,NaN,NaN
3,1.0,18-24,CONSTRUCTION,Others,51885.0,BACHELOR OF ARTS
4,6.0,25-34,NaN,NaN,62400.0,SOME COLLEGE (NO DEGREE)
5,0.0,< 18,FOOD AND BEVERAGE,WAITERS AND WAITRESSES,15169.0,ASSOCIATE OF ARTS
6,4.0,18-24,NaN,RETAIL SALESPERSONS,26325.0,Others
7,1.0,18-24,NaN,NaN,41447.0,NaN
8,0.0,18-24,CHILDCARE,NANNIES,28452.0,NaN
9,0.0,18-24,CUSTOMER SERVICE,"COMMUNITY AND SOCIAL SERVICE SPECIALISTS, ALL ...",16331.0,NaN


In [148]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

df = pd.DataFrame({'A':['a','b','a',np.nan,'a','b','c'] , 'B':[1,2,4,np.NaN,6,7,9] , 'C':['q','e',np.nan,'w','w','e','q'],'D':['q','e','o','w','w','e','q']})  

mm = MinMaxScaler()

def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].dropna().unique(), 0)}

def integer_encode(df , variable, ordinal_mapping):
    df[variable] = df[variable].map(ordinal_mapping)
    

mappin = dict()
def imputation(df1 , cols):
    df = df1.copy()
    #Encoding dict &amp; Removing nan    
    #mappin = dict()
    for variable in cols:
        mappings = find_category_mappings(df, variable)
        mappin[variable] = mappings


    #Apply mapping
    for variable in cols:
        integer_encode(df, variable, mappin[variable])  

    
    #Minmaxscaler and KNN imputation 
    sca = mm.fit_transform(df)
    knn_imputer = KNNImputer()
    knn = knn_imputer.fit_transform(sca)
    df.iloc[:,:] = mm.inverse_transform(knn)
    for i in df.columns : 
        df[i] = round(df[i]).astype('float')

    #Inverse transform
    for i in cols:
        inv_map = {v: k for k, v in mappin[i].items()}
        df[i] = df[i].map(inv_map)
    return df


#Imputation
j=imputation(df,['A','C','D'])
j

,A,B,C,D
0,a,1.0,q,q
1,b,2.0,e,e
2,a,4.0,e,o
3,a,4.0,w,w
4,a,6.0,w,w
5,b,7.0,e,e
6,c,9.0,q,q


In [149]:
j.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       7 non-null      object 
 1   B       7 non-null      float64
 2   C       7 non-null      object 
 3   D       7 non-null      object 
dtypes: float64(1), object(3)
memory usage: 352.0+ bytes


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       6 non-null      object 
 1   B       6 non-null      float64
 2   C       6 non-null      object 
 3   D       7 non-null      object 
dtypes: float64(1), object(3)
memory usage: 352.0+ bytes
